# Optimization II 
## HW2


In [1]:
import numpy as np
import random
import pandas as pd
import gurobipy as gp

### Problem 1

In [2]:
#read csv
stocks2019 = pd.read_csv(r'.\stocks2019.csv')
stocks2020 = pd.read_csv(r'.\stocks2020-1.csv')

In [3]:
nstocks = stocks2019.shape[1]-2
ndays = stocks2019.shape[0]
ret2019 = stocks2019.iloc[1:ndays,2:nstocks+2].values - stocks2019.iloc[0:ndays-1,2:nstocks+2].values
ret2019 = np.divide(ret2019,stocks2019.iloc[0:ndays-1,2:nstocks+2].values)
ndays = ret2019.shape[0]

In [4]:
ndays2020 = stocks2020.shape[0]
ret2020 = stocks2020.iloc[1:ndays2020,2:nstocks+2].values - stocks2020.iloc[0:ndays2020-1,2:nstocks+2].values
ret2020 = np.divide(ret2020,stocks2020.iloc[0:ndays2020-1,2:nstocks+2].values)
ndays2020 = ret2020.shape[0]

meanvec = np.mean(ret2019,0)

In [5]:
c = np.concatenate((meanvec,[0]*ndays))
A = np.zeros((2+ndays,nstocks+ndays))
b = np.zeros((2+ndays,1))

A[0,0:nstocks] = 1 # all money invested
b[0] = 1

# no more than 100 days negative
A[1,nstocks:nstocks+ndays] = 1
b[1] = 100

M = 10
for day in range(1,ndays+1):
    A[1+day,0:nstocks] = ret2019[day-1,0:nstocks]
    A[1+day,nstocks+day-1] = M
    b[1+day] = 0
    

sense = np.concatenate((['='],['<'],['>']*ndays))

In [6]:
m = gp.Model()
mX = m.addMVar(len(c),vtype = np.concatenate((['C']*nstocks,['B']*ndays)))
m.setMObjective(None, c, 0, sense = gp.GRB.MAXIMIZE)
m.addMConstrs(A, mX, sense, b)
m.Params.OutputFlag = 1

#Solve the IP using gurobi 
m.optimize()

# geez, AMD had a bonkers year in 2019, and an even more bonkers year in 2020

Academic license - for non-commercial use only - expires 2022-09-13
Using license file C:\Users\suyas\gurobi.lic
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 252 rows, 350 columns and 25504 nonzeros
Model fingerprint: 0x10b2ab9f
Variable types: 100 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+01]
  Objective range  [7e-05, 4e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.07s
Presolved: 251 rows, 349 columns, 25402 nonzeros
Variable types: 100 continuous, 249 integer (249 binary)

Root relaxation: objective 4.097099e-03, 112 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

In [7]:
num_neg = 0
for day in range(1,ndays2020+1):
    port_ret = sum(np.multiply(mX.x[0:nstocks],ret2020[day-1,0:nstocks]))
    if(port_ret < 0):
        num_neg = num_neg + 1
        
print(num_neg)

115


### Problem 2

In [8]:
# let hij = profit on day i for newspaper j
# decision variables are (q1, q2, q3, q4, q5, h11, h12, ..., h150, h21, ..., h250, ... , h550)

demand = pd.read_csv(r'.\hw2_demand.csv')
nnews = demand.shape[1]
ndays = demand.shape[0]

In [9]:
costs = np.array([0.5,0.5,0.25,0.75,0.25])
prices = np.array([1.25,1.2,0.5,1.5,0.6])

In [10]:
c = np.concatenate(([0]*nnews,[1/ndays]*ndays*nnews))
lbound = np.concatenate(([0]*nnews,[-np.Inf]*ndays*nnews))

A = np.zeros((2*ndays*nnews,ndays*nnews+nnews))
b = np.zeros((2*ndays*nnews,1))
sense = ['<']*2*ndays*nnews

In [11]:
for news in range(1,nnews+1):
    for day in range(1,ndays+1):
        this_nd = day + (news-1)*ndays
        A[2*this_nd-1-1,news-1] = costs[news-1]
        A[2*this_nd-1-1,this_nd+nnews-1] = 1
        b[2*this_nd-1-1] = prices[news-1]*demand.iloc[day-1,news-1]
        A[2*this_nd-1,news-1] = costs[news-1]-prices[news-1]
        A[2*this_nd-1,this_nd+nnews-1] = 1

In [12]:
m = gp.Model()
mX = m.addMVar(len(c),vtype = ['C']*len(c),lb = lbound)
m.setMObjective(None, c, 0, sense = gp.GRB.MAXIMIZE)
m.addMConstrs(A, mX, sense, b)
m.Params.OutputFlag = 1

m.optimize()

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 500 rows, 255 columns and 1000 nonzeros
Model fingerprint: 0x44ab24b6
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e-02, 2e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 250 rows and 0 columns
Presolve time: 0.02s
Presolved: 250 rows, 255 columns, 500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4081840e+03   7.040920e+04   0.000000e+00      0s
     115    3.3698900e+02   0.000000e+00   0.000000e+00      0s

Solved in 115 iterations and 0.02 seconds
Optimal objective  3.369890000e+02


In [13]:
print(mX.x[0:nnews]) # how many we should produce
print(m.objval) # expectation of profits given that many produced

[ 76. 462. 205. 123. 422.]
336.9890000000003


### Problem 3

In [14]:
nsim = 10000
def_vec = [False]*nsim
port_val_vec = [0]*nsim
years_working = 3 # how many years are you going to stay employed?
# I just ran this for a few different values of years.working, and 3 was the smallest number of years with less than 6% chance of default
# it's important that each time you re-run it for different values of years.working, you also reset def.vec and port.val.vec!!!!!!!!

for sim in range(1,nsim+1):
    value = 2000000
    current_salary = 400000 # at the beginning of each sim you current salary is 400k
    for year in range(1,36):
        port_return = np.random.normal(size = 1, loc = 0.07, scale = 0.1)
        value = value*(1+port_return)
        value = value - 150000
        if(year <= years_working): # are you still working?
            value = value + current_salary # add your salary back to your account (you already removed the amount you spend)
            current_salary = current_salary*1.025 # increase your salary for next year
        if(value <= 0):
            value = 0
    if(value <= 0):
        def_vec[sim-1] = True
    port_val_vec[sim-1] = value

np.mean(def_vec)

0.0505

### Problem 4

In [15]:
nsim = 10000
profit_vec = [0]*nsim # vector to hold each simulation's profit
for sim in range(1,nsim+1): # loop through the simulations
    chipsAperf = np.random.normal(size = 100, loc = 10, scale = 3) # get performance of all chips manufactured using first method
    chipsBperf = np.random.normal(size = 200, loc = 5, scale = 2) # get performance of all chips manufactured using second method
    profitA = [0]*100 # initialize profits of all first method chips to be zero (we'll fix this in the next few lines)
    for i in range(100):
        if(chipsAperf[i] >= 7): # set profits of high performance chips
            profitA[i] = 15-10
        else:
            profitA[i] = 8-10 # set profits (or losses) of low performance chips
            
    profitB = [0]*200 # initialize profits of all second method chips to be zero (we'll fix this in the next few lines)
    for i in range(200):
        if(chipsBperf[i] >= 7):
            profitB[i] = 15-5 # set profits of high performance chips
        else:
            profitB[i] = 8-5 # set profits of low performance chips
            
    profit_vec[sim-1] = sum(profitA)+sum(profitB) # the total profit in this simulation is the sum of all profits

np.quantile(profit_vec,np.array([0.25,0.975])) # use quantile to get 95% predictive/confidence interval

array([1184., 1296.])

In [16]:
np.std(profit_vec) # use std to get the standard deviation

43.721068892583126